In [1]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple
import torch.optim as optim
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from datasets import load_dataset
import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import Strategy
from flwr.simulation import run_simulation
# from flwr_datasets import FederatedDataset
from typing import Union
from flwr.server.client_proxy import ClientProxy  # Correctly import ClientProxy
from flwr.common import FitRes, Parameters
from torch.optim import lr_scheduler
from sklearn.preprocessing import LabelEncoder
import glob
import os
from typing import Union
from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder
from datasets import load_dataset
from torch.utils.data import DataLoader
import pandas as pd
from flwr_datasets.partitioner import IidPartitioner
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg
from flwr.server.strategy import FedAvg
DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")

/home/stagiaire-stack/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-03 09:11:01,853	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Training on cpu
Flower 1.17.0 / PyTorch 2.6.0+cu124


/home/stagiaire-stack/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:

NUM_CLIENTS = 15
BATCH_SIZE = 64
MAX_ROUND = 10
NUM_PARTITIONS = NUM_CLIENTS

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE


In [ ]:

from datasets import load_dataset
from torch.utils.data import DataLoader
from flwr_datasets.partitioner import IidPartitioner
from flwr_datasets import FederatedDataset
class AttackDataset(Dataset):
    def __init__(self, hf_dataset):
        self.dataset = hf_dataset
        self.feature_keys = [k for k in hf_dataset.column_names if k != "Attack_type"]

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
            item = self.dataset[idx]
            X = torch.tensor([item[k] for k in self.feature_keys], dtype=torch.float32)
            y = torch.tensor(item["Attack_type"], dtype=torch.long)
            return X, y


def load_datasets(partition_id: int, num_partitions: int, min_dataset_size=50):
    data_files = {
        "train": "preprocessed_DNN-EdgeIIoT-dataset.csv"
    }

    

    # Charge tout le DatasetDict
    dataset_dict = load_dataset("csv", data_files=data_files)
    dataset_dict = dataset_dict.remove_columns(["Attack_label"])
    # Prend uniquement le split 'train' pour le partitionnement
    train_dataset = dataset_dict["train"]

    # Partitionnement
    partitioner = IidPartitioner(num_partitions=num_partitions)
    partitioner.dataset = train_dataset

    client_dataset = partitioner.load_partition(partition_id)

    if len(client_dataset) < min_dataset_size:
        raise ValueError(f"Partition {partition_id} trop petite ({len(client_dataset)} échantillons), minimum requis = {min_dataset_size}")

    # Split local train/val/test : 70% / 15% / 15%
    train_val_test = client_dataset.train_test_split(test_size=0.3, seed=42)
    val_test = train_val_test["test"].train_test_split(test_size=0.5, seed=42)

    train_set = AttackDataset(train_val_test["train"])
    val_set = AttackDataset(val_test["train"])
    test_set = AttackDataset(val_test["test"])

    # DataLoaders
    trainloader = DataLoader(train_set, batch_size=32, shuffle=True, num_workers=4)
    valloader = DataLoader(val_set, batch_size=32, shuffle=False, num_workers=4)
    testloader = DataLoader(test_set, batch_size=32, shuffle=False, num_workers=4)

    return trainloader, valloader, testloader


In [5]:


class MLP(nn.Module):
    def __init__(self, input_size=96, num_classes=15):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 90),
            nn.ReLU(),
            nn.Linear(90, 90),
            nn.ReLU(),
            nn.Linear(90, num_classes)  # output logits pour chaque classe
        )

    def forward(self, x):
        return self.layers(x)

In [6]:


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

def train(net, trainloader, epochs: int):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-5)  # L2 régularisation ici    
    scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[50, 75], gamma=0.1)

    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for X, y in trainloader:
            X, y = X.to(DEVICE), y.to(DEVICE)  # y doit rester en long pour CrossEntropy

            optimizer.zero_grad()
            outputs = net(X)  # shape: [B, 15]
            outputs = F.softmax(outputs, dim=1)

            # print("outputs")
            # print(outputs)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()



            # Prédictions
            correct += (outputs.argmax(dim=1) == y).sum().item()
            total += y.size(0)
            epoch_loss += loss.item()

        epoch_loss /= len(trainloader)
        epoch_acc = correct / total
        scheduler.step()

        print(f"Epoch {epoch+1}: train loss {epoch_loss:.4f}, accuracy {epoch_acc:.4f}")


def test(net, testloader):
    criterion = nn.CrossEntropyLoss()
    correct, total, total_loss = 0, 0, 0.0
    net.eval()

    with torch.no_grad():
        for X, y in testloader:
            X, y = X.to(DEVICE), y.to(DEVICE)

            outputs = net(X)  # shape: [B, 15]
            _, predicted = outputs.max(1)

            loss = criterion(outputs, y)
            total_loss += loss.item()

            # _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == y).sum().item()
            total += y.size(0)

    avg_loss = total_loss / len(testloader)
    accuracy = correct / total

    return avg_loss, accuracy


In [7]:
class FlowerClient(NumPyClient):
    def __init__(self, partition_id, net, trainloader, valloader):
        self.partition_id = partition_id
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.partition_id}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.partition_id}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=3)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.partition_id}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(context: Context) -> Client:
    net = MLP().to(DEVICE)
    partition_id = context.node_config["partition-id"]
    num_partitions = context.node_config["num-partitions"]
    trainloader, valloader, _ = load_datasets(partition_id, num_partitions)
    return FlowerClient(partition_id, net, trainloader, valloader).to_client()


# Create the ClientApp
client = ClientApp(client_fn=client_fn)

In [8]:
import flwr as fl

net = MLP()

class SaveModelStrategy(fl.server.strategy.FedAvg):
    def aggregate_fit(
        self,
        server_round: int,
        results: list[tuple[fl.server.client_proxy.ClientProxy, fl.common.FitRes]],
        failures: list[Union[tuple[ClientProxy, FitRes], BaseException]],
    ) -> tuple[Optional[Parameters], dict[str, Scalar]]:
        """Aggregate model weights using weighted average and store checkpoint"""
        print(f"Results from clients: {results}")  # Debugging line
        # Call aggregate_fit from base class (FedAvg) to aggregate parameters and metrics
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(
            server_round, results, failures
        )

        if server_round == MAX_ROUND:

            if aggregated_parameters is not None:
                print(f"Saving round {server_round} aggregated_parameters...")

                # Convert `Parameters` to `list[np.ndarray]`
                aggregated_ndarrays: list[np.ndarray] = fl.common.parameters_to_ndarrays(
                    aggregated_parameters
                )
                # Convert `list[np.ndarray]` to PyTorch `state_dict`
                params_dict = zip(net.state_dict().keys(), aggregated_ndarrays)
                state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
                net.load_state_dict(state_dict, strict=True)

                # Save the model to disk
                torch.save(net.state_dict(), f"model/model_round_{server_round}.pth")

        return aggregated_parameters, aggregated_metrics

In [9]:
from flwr.common import NDArrays
def evaluate(
    server_round: int,
    parameters: NDArrays,
    config: Dict[str, Scalar],
) -> Optional[Tuple[float, Dict[str, Scalar]]]:
    net = MLP().to(DEVICE)
    _, _, testloader = load_datasets(0, NUM_PARTITIONS)
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(net, testloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

In [10]:
# Create strategy and pass into ServerApp
def server_fn(context):
    strategy = SaveModelStrategy(
        fraction_fit=1.0,  # Utiliser 100% des clients pour l'entraînement
        fraction_evaluate=1.0,  # Utiliser 10% des clients pour l'évaluation
        min_fit_clients=NUM_CLIENTS,  # Minimum de 10 clients pour l'entraînement
        min_evaluate_clients=NUM_CLIENTS,  # Minimum de 5 clients pour l'évaluation
        min_available_clients=NUM_CLIENTS,
        evaluate_fn=evaluate
    )
    
    
    config = ServerConfig(num_rounds=MAX_ROUND)
    return ServerAppComponents(strategy=strategy, config=config)




In [11]:
app = ServerApp(server_fn=server_fn)

server = ServerApp(server_fn=server_fn)
backend_config = {"client_resources": None}


run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)


INFO :      Starting Flower ServerApp, config: num_rounds=10, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(ClientAppActor pid=139062) /home/stagiaire-stack/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
(ClientAppActor pid=139062)   warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
INFO :      Received initial parameters from one random client
INFO :      Starting evaluation of initial global parameters


(ClientAppActor pid=139062) [Client 7] get_parameters


ERROR :     ServerApp thread raised an exception: mat1 and mat2 shapes cannot be multiplied (32x97 and 96x90)
ERROR :     Traceback (most recent call last):
  File "/home/stagiaire-stack/.local/lib/python3.10/site-packages/flwr/simulation/run_simulation.py", line 268, in server_th_with_start_checks
    updated_context = _run(
  File "/home/stagiaire-stack/.local/lib/python3.10/site-packages/flwr/server/run_serverapp.py", line 62, in run
    server_app(grid=grid, context=context)
  File "/home/stagiaire-stack/.local/lib/python3.10/site-packages/flwr/server/server_app.py", line 142, in __call__
    start_grid(
  File "/home/stagiaire-stack/.local/lib/python3.10/site-packages/flwr/server/compat/app.py", line 90, in start_grid
    hist = run_fl(
  File "/home/stagiaire-stack/.local/lib/python3.10/site-packages/flwr/server/server.py", line 492, in run_fl
    hist, elapsed_time = server.fit(
  File "/home/stagiaire-stack/.local/lib/python3.10/site-packages/flwr/server/server.py", line 95, in

RuntimeError: Exception in ServerApp thread